In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 14871844
paper_name = 'aouida_ramotar_2004' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_excel('raw_data/CAN_2-1-04_Aouida.xlsx', sheet_name='Sheet1', header=None)

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 236 x 2


In [10]:
original_data.head()

,0,1
0,YBR101C,R+++
1,YJR059W,R+++
2,YBR132C,R++++
3,YGL007W,R++++
4,YMR216C,R++++


In [11]:
original_data['orf'] = original_data[0].astype(str)

In [12]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [0, 1, orf]
Index: []


In [15]:
data_switch = {'R+++': 9, 'R++++': 10, 'S+': -1, 'S++': -2, 'S+++': -3, 'S++++': -4}

In [17]:
original_data['data'] = original_data[1].apply(lambda x: data_switch[x.strip('\xa0')])

In [18]:
original_data.set_index('orf', inplace=True)

In [19]:
original_data = original_data[['data']].copy()

In [20]:
original_data = original_data.groupby(original_data.index).mean()

In [21]:
original_data.shape

(236, 1)

# Load & process tested strains

In [25]:
tested = pd.read_excel('raw_data/HU haploid.xlsx', sheet_name='Sheet1', skiprows=2)

In [26]:
tested.head()

,Plate,row,col,record #,ORF,strain,Euroscarf comments,Euroscarf growth comment,Freezing comments,Duplication in our collection,Unnamed: 10,Single on HU,gene name(s),brief id (fromSGD)
0,1,A,1,338.0,YAL068C,BY4741,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"biological_process unknown, molecular_function..."
1,1,A,2,339.0,YAL067C,BY4741,NaN,NaN,NaN,NaN,NaN,NaN,SEO1,putative permease
2,1,A,3,340.0,YAL066W,BY4741,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"biological_process unknown, molecular_function..."
3,1,A,4,341.0,YAL065C,BY4741,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"biological_process unknown, molecular_function..."
4,1,A,5,345.0,YAL062W,BY4741,NaN,NaN,NaN,NaN,NaN,NaN,"GDH3, FUN51","NADP-linked glutamate dehydrogenase,glutamate ..."


In [27]:
tested['orf'] = tested['ORF'].astype(str)

In [28]:
tested['orf'] = clean_orf(tested['orf'])

In [29]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [30]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

             Plate row  col  record #  ORF strain Euroscarf comments  \
index_input                                                            
1524            16   H    1       NaN  NaN    NaN                NaN   
1525            16   H    2       NaN  NaN    NaN                NaN   
1526            16   H    3       NaN  NaN    NaN                NaN   
1527            16   H    4       NaN  NaN    NaN                NaN   
1528            16   H    5       NaN  NaN    NaN                NaN   
1529            16   H    6       NaN  NaN    NaN                NaN   
1530            16   H    7       NaN  NaN    NaN                NaN   
1531            16   H    8       NaN  NaN    NaN                NaN   
1532            16   H    9       NaN  NaN    NaN                NaN   
1533            16   H   10       NaN  NaN    NaN                NaN   
1534            16   H   11       NaN  NaN    NaN                NaN   
1535            16   H   12       NaN  NaN    NaN               

In [31]:
tested = tested.loc[t,:]

In [32]:
tested_orfs = tested['orf'].unique()

In [33]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

[]

In [34]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

# Prepare the final dataset

In [35]:
data = original_data.copy()

In [36]:
dataset_ids = [96]
datasets = datasets.reindex(index=dataset_ids)

In [37]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [38]:
data.head()

dataset_id,96
data_type,value
orf,
YAL068C,0
YAL067C,0
YAL066W,0
YAL065C,0
YAL062W,0


## Subset to the genes currently in SGD

In [39]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [40]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,96
,data_type,value
gene_id,orf,
1869,YAL068C,0
61,YAL067C,0
60,YAL066W,0
1727,YAL065C,0
57,YAL062W,0


# Normalize

In [41]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [42]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [43]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id         96          
data_type       value    valuez
gene_id orf                    
1869    YAL068C     0  0.052724
61      YAL067C     0  0.052724
60      YAL066W     0  0.052724
1727    YAL065C     0  0.052724
57      YAL062W     0  0.052724

# Print out

In [44]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [45]:
from IO.save_data_to_db3 import *

In [46]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 14871844...
Inserting the new data...


100%|██████████| 1/1 [00:08<00:00,  8.08s/it]

Updating the data_modified_on field...
